In [2]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from scipy.optimize import minimize

# 1. Prepare the Data (Function 7, Index 6)
# Inputs from Week 1, 2, 3
X_train = np.array([
    [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],                         # Week 1
    [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],                         # Week 2
    [0.213787, 0.185417, 0.205796, 0.191078, 0.174785, 0.191819] # Week 3
])

# Outputs from Week 1, 2, 3
y_train = np.array([
    3.408539273427753e-05,
    0.4081092157295297,
    0.34688512628942264
])

# 2. Define the Kernel
# Matern kernel is generally better for physical functions than RBF.
# WhiteKernel handles noise in the observations.
kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=1e-5)

# 3. Fit the Gaussian Process
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
gp.fit(X_train, y_train)

# 4. Define Acquisition Function (Upper Confidence Bound)
# We want to maximize the function. UCB = mu + kappa * sigma
def acquisition_func(x):
    x = x.reshape(1, -1)
    mean, std = gp.predict(x, return_std=True)
    kappa = 1.96  # Exploration/Exploitation balance parameter
    return -(mean + kappa * std) # Negate because scipy minimizes

# 5. Optimize to find the next input
# We start the search around the best point found so far (Week 2's 0.2)
best_x_so_far = X_train[np.argmax(y_train)]
bounds = [(0.01, 1.0)] * 6  # Constraint: Positive numbers only

res = minimize(
    acquisition_func, 
    x0=best_x_so_far, 
    bounds=bounds, 
    method='L-BFGS-B'
)

# 6. Result
next_input = res.x
print("Recommended Submission for Week 4 (Function 7):")
print(np.array2string(next_input, separator=', '))

# Optional: Predict expected value for this input
pred_mean, pred_std = gp.predict(next_input.reshape(1, -1), return_std=True)
print(f"Predicted Output: {pred_mean[0]:.4f} (std: {pred_std[0]:.4f})")

Recommended Submission for Week 4 (Function 7):
[0.0832023 , 0.32354808, 0.15089491, 0.27558739, 0.41362356, 0.26930961]
Predicted Output: 0.2909 (std: 0.2082)
